In [1]:
import time
import psycopg2
import sqlite3
import numpy

In [2]:
def getz(ra, dec):
    start_time = time.time()
    f = open('/global/cfs/cdirs/desi/science/td/secrets/desi_pg.txt') 
    file = f.read()
    db_name, db_user, db_pwd, db_host = file.split()
    db_name = 'desidb'
    conn = psycopg2.connect(dbname=db_name, user=db_user, password=db_pwd, host=db_host)
    cur = conn.cursor()
    curr = """SELECT f.targetid,c.tileid,c.night,r.z, f.target_ra, f.target_dec, r.zerr,r.zwarn,c.filename
FROM daily.tiles_fibermap f
INNER JOIN daily.cumulative_tiles c ON f.cumultile_id=c.id
INNER JOIN daily.tiles_redshifts r ON r.cumultile_id=c.id AND r.targetid=f.targetid
WHERE q3c_radial_query( f.target_ra, f.target_dec, {0}, {1}, 10./3600. )""".format(ra, dec)
    cur.execute(curr) 
    desi_arr = cur.fetchall()
    
    if (len(desi_arr) == 0):
        curr = """SELECT f.targetid,c.tileid,c.night,r.z, f.target_ra, f.target_dec, r.zerr,r.zwarn,c.filename
FROM fuji.tiles_fibermap f
INNER JOIN fuji.cumulative_tiles c ON f.cumultile_id=c.id
INNER JOIN fuji.tiles_redshifts r ON r.cumultile_id=c.id AND r.targetid=f.targetid
WHERE q3c_radial_query( f.target_ra, f.target_dec, {0}, {1}, 10./3600. )""".format(ra, dec)
        cur.execute(curr) 
        desi_arr = cur.fetchall()

    if (len(desi_arr) == 0):
        curr = """SELECT f.targetid,c.tileid,c.night,r.z, f.target_ra, f.target_dec, r.zerr,r.zwarn,c.filename
FROM guadalupe.tiles_fibermap f
INNER JOIN guadalupe.cumulative_tiles c ON f.cumultile_id=c.id
INNER JOIN guadalupe.tiles_redshifts r ON r.cumultile_id=c.id AND r.targetid=f.targetid
WHERE q3c_radial_query( f.target_ra, f.target_dec, {0}, {1}, 10./3600. )""".format(ra, dec)
        cur.execute(curr) 
        desi_arr = cur.fetchall()
        

    # if (len(desi_arr) == 0):
    #     print("No information available!")
    #     cur.close()
    #     conn.close()    
    # else:
    #     cur.close()
    #     conn.close()
    #     z = desi_arr[0][3]
    #     if (z > 0.15):
    #         print("To be provided:")
    #     elif (z < 0.15):
    #         print("Not to be provided:")
    #     return desi_arr
    
    cur.close()
    conn.close()
    return desi_arr

In [21]:
def helper(input_info):
    #helper_info is a dic of ZTF id, input_ra, input_dec
    
    ZTFids = input_info['id']
    ras = input_info['RA']
    decs = input_info['DEC']
    print("ZTF ID" + "\t \t Input RA" + "\t Input DEC" + "\t DESI TID" + "\t     DESI Night" + "\t \t DESI Z" + "\t \t \t DESI RA" + "\t \t DESI DEC")
    for i in range(len(ras)):
        ra = ras[i]
        dec = decs[i]
        ztfid = ZTFids[i]
        
        zinfo = getz(ra, dec)
        
        if (len(zinfo) == 0):
            print(str(ztfid) + "\t" +  str(ra) + "\t" + str(dec) + "\t No Info." )
        else:
            tid = zinfo[0][0]
            night = zinfo[0][2]
            z = zinfo[0][3]
            desi_ra = zinfo[0][4]
            desi_dec =zinfo[0][5]
            if (z > 0.15):
                print(str(ztfid) + "\t" +  str(ra) + "\t" + str(dec) + "\t" + str(tid) + "\t"  + str(night) + "\t" + str(z) + "\t" + str(desi_ra)+ "\t" + str(desi_dec))
                
            elif (z < 0.15):
                print(str(ztfid) + "\t" +  str(ra) + "\t" + str(dec) + "\t" + str(tid) + "\t"  + str(night) + "\t N/A" )
        

In [22]:
data = {'id': ['ZTF19abiptrq', 'ZTF20aaodnpo', 'ZTF19abdkdze', 'ZTF19abpfedt', 'ZTF20aatqgob', 'ZTF20aandifv', 'ZTF20abatzio', 'ZTF20aajbzmc', 'ZTF20abeoibi'], 
        'RA': [357.1602996, 250.8160264, 20.5682466, 3.6635952, 206.5802377, 156.808529, 176.1658846, 187.4783937, 219.2628527], 
        'DEC' : [19.2628105, 7.0175426, 12.7875204, 19.7418401,  15.56176,  80.2705631,  68.8570115, 16.188751, 26.7827165]}

In [23]:
helper(data)

ZTF ID	 	 Input RA	 Input DEC	 DESI TID	     DESI Night	 	 DESI Z	 	 	 DESI RA	 	 DESI DEC
ZTF19abiptrq	357.1602996	19.2628105	 No Info.
ZTF20aaodnpo	250.8160264	7.0175426	39627957735524628	20220511	0.154196114463356	250.815946841308	7.017419911679
ZTF19abdkdze	20.5682466	12.7875204	 No Info.
ZTF19abpfedt	3.6635952	19.7418401	 No Info.
ZTF20aatqgob	206.5802377	15.56176	 No Info.
ZTF20aandifv	156.808529	80.2705631	 No Info.
ZTF20abatzio	176.1658846	68.8570115	 No Info.
ZTF20aajbzmc	187.4783937	16.188751	 No Info.
ZTF20abeoibi	219.2628527	26.7827165	 No Info.


In [45]:
getz(250.8160264, 7.0175426)

To be provided:


0.154196114463356

In [46]:
getz(3.6635952, 19.7418401)

No information available!


In [63]:
def getz(ra, dec):
    start_time = time.time()
    f = open('/global/cfs/cdirs/desi/science/td/secrets/desi_pg.txt')
    file = f.read()
    db_name, db_user, db_pwd, db_host = file.split()
    db_name = 'desidb'
    conn = psycopg2.connect(dbname=db_name, user=db_user, password=db_pwd, host=db_host)
    cur = conn.cursor()
    curr = """SELECT f.targetid,c.tileid,c.night,r.z,r.zerr,r.zwarn,c.filename
FROM daily.tiles_fibermap f
INNER JOIN daily.cumulative_tiles c ON f.cumultile_id=c.id
INNER JOIN daily.tiles_redshifts r ON r.cumultile_id=c.id AND r.targetid=f.targetid
WHERE q3c_radial_query( f.target_ra, f.target_dec, {0}, {1}, 10./3600. )

UNION
SELECT f.targetid,c.tileid,c.night,r.z,r.zerr,r.zwarn,c.filename
FROM fuji.tiles_fibermap f
INNER JOIN fuji.cumulative_tiles c ON f.cumultile_id=c.id
INNER JOIN fuji.tiles_redshifts r ON r.cumultile_id=c.id AND r.targetid=f.targetid
WHERE q3c_radial_query( f.target_ra, f.target_dec, {0}, {1}, 10./3600. )

UNION
SELECT f.targetid,c.tileid,c.night,r.z,r.zerr,r.zwarn,c.filename
FROM guadalupe.tiles_fibermap f
INNER JOIN guadalupe.cumulative_tiles c ON f.cumultile_id=c.id
INNER JOIN guadalupe.tiles_redshifts r ON r.cumultile_id=c.id AND r.targetid=f.targetid
WHERE q3c_radial_query( f.target_ra, f.target_dec, {0}, {1}, 10./3600. )""".format(ra, dec)
    cur.execute(curr) 
    desi_arr = cur.fetchall()    

    if (len(desi_arr) == 0):
        print("No information available!")
        cur.close()
        conn.close()    
    else:
        cur.close()
        conn.close()
        z = desi_arr[0][3]
        if (z > 0.15):
            print("To be provided:")
        elif (z < 0.15):
            print("Not to be provided:")
        return desi_arr

In [64]:
getz(250.8160264  ,   7.0175426)

To be provided:


[(39627957735524628,
  21976,
  20220511,
  0.154196114463356,
  7.98915799003484e-06,
  0,
  'daily/tiles/cumulative/21976/20220511/redrock-7-21976-thru20220511.fits')]

In [62]:
getz(3.6635952, 19.7418401)

No information available!
